<a href="https://colab.research.google.com/github/Erick88santos/ORACLE-DBA-SCRIPTS-AND-TROUBLESHOTINGS/blob/main/configurar_o_statspack_no_oracle_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Configurar o Statspack no Oracle Database**

Para configurar o Statspack no Oracle Database, siga estas etapas:
#1.	**Conectar-se como DBA:** Utilize um cliente SQL ou a linha de comando para se conectar ao Oracle Database como um usuário com privilégios de DBA (Database Administrator).



#2.	 **Criar a Tabela de Estatísticas:** Execute o script spcreate.sql, localizado no diretório $ORACLE_HOME/rdbms/admin, para criar a tabela de estatísticas do Statspack. Este script cria a tabela STATSPACK no schema selecionado.

``@?/rdbms/admin/spcreate.sql``



#3. **Configurar o Schema de Coleta de Estatísticas:** Defina o schema onde as estatísticas do Statspack serão armazenadas. Isso é feito alterando a variável de ambiente STATSOWN para o nome do schema desejado. Por exemplo, para definir o schema como **STATPACK:**

``ALTER SESSION SET EVENTS '10401 TRACE NAME CONTEXT FOREVER, LEVEL 1';``

``ALTER SYSTEM SET STATISTICS_LEVEL = TYPICAL SCOPE = BOTH;``



#4. **Executar a Coleta de Estatísticas:** Para iniciar a coleta de estatísticas, execute o procedimento statspack.snap especificando o intervalo de coleta desejado. Por exemplo, para coletar estatísticas a cada hora:

``EXEC statspack.snap;``



#5. **Gerar Relatórios de Desempenho:** Para gerar um relatório de desempenho com base nas estatísticas coletadas, execute o procedimento *statspack.report.* Por exemplo, para gerar um relatório para o último snapshot:

``SET LONG 1000000``
``SET PAGESIZE 1000``
``SET LINESIZE 200``
``SELECT * FROM TABLE(statspack.snap);``



#6. **Automatizar a Coleta de Estatísticas (Opcional):** Você pode configurar um job no Oracle Scheduler para executar o procedimento statspack.snap periodicamente e automatizar a coleta de estatísticas.

Lembre-se de revisar a documentação do Oracle Database e adaptar essas instruções de acordo com a sua versão específica do banco de dados e requisitos do seu ambiente.

#*Para automatizar a coleta de estatísticas com o Statspack no Oracle Database, você pode usar o Oracle Scheduler para agendar a execução do procedimento `statspack.snap` em intervalos regulares. Aqui estão os passos para criar essa automação:*



#1. **Criar um Job no Oracle Scheduler**: Use o seguinte comando SQL para criar um job no Oracle Scheduler que executará o procedimento `statspack.snap` a cada hora:

   ```sql
   BEGIN
     DBMS_SCHEDULER.create_job (
       job_name        => 'COLETA_STATS',
       job_type        => 'PLSQL_BLOCK',
       job_action      => 'BEGIN statspack.snap; END;',
       start_date      => SYSTIMESTAMP,
       repeat_interval => 'FREQ=HOURLY',
       enabled         => TRUE
     );
   END;
   /
   ```

   Neste exemplo, o job é criado com o nome `COLETA_STATS`, e o procedimento `statspack.snap` é executado a cada hora.



#2. **Verificar a Configuração do Job**: Você pode verificar se o job foi criado corretamente consultando a view `DBA_SCHEDULER_JOBS`. Por exemplo:

   ```sql
   SELECT job_name, enabled FROM DBA_SCHEDULER_JOBS WHERE job_name = 'COLETA_STATS';
   ```

   Certifique-se de que o job esteja habilitado (`ENABLED = TRUE`).



#3. **Monitorar a Execução do Job**: Você pode monitorar a execução do job verificando a view `DBA_SCHEDULER_JOB_RUN_DETAILS`. Por exemplo, para ver os últimos 10 históricos de execução do job:

   ```sql
   SELECT * FROM DBA_SCHEDULER_JOB_RUN_DETAILS WHERE job_name = 'COLETA_STATS' ORDER BY log_date DESC FETCH FIRST 10 ROWS ONLY;
   ```



#4. **Parar ou Excluir o Job (Opcional)**: Se desejar parar ou excluir o job no futuro, você pode usar os seguintes comandos:

   - Para parar o job:

     ```sql
     BEGIN
       DBMS_SCHEDULER.disable('COLETA_STATS');
     END;
     /
     ```

   - Para excluir o job:

     ```sql
     BEGIN
       DBMS_SCHEDULER.drop_job('COLETA_STATS');
     END;
     /
     ```

Esses são os passos básicos para automatizar a coleta de estatísticas com o Statspack usando o Oracle Scheduler. Certifique-se de ajustar a frequência e outros parâmetros conforme necessário para atender aos requisitos do seu ambiente.


Para criar um job no Oracle Scheduler que execute o procedimento `statspack.snap` a cada 3 minutos, você pode usar o seguinte comando SQL:

```sql
BEGIN
  DBMS_SCHEDULER.create_job (
    job_name        => 'COLETA_STATS',
    job_type        => 'PLSQL_BLOCK',
    job_action      => 'BEGIN statspack.snap; END;',
    start_date      => SYSTIMESTAMP,
    repeat_interval => 'FREQ=MINUTELY; INTERVAL=3',
    enabled         => TRUE
  );
END;
/
```

Neste exemplo, o job é criado com o nome `COLETA_STATS`, e o procedimento `statspack.snap` é executado a cada 3 minutos. Certifique-se de que o job esteja habilitado (`ENABLED = TRUE`).

Para verificar se o job foi criado corretamente, você pode consultar a view `DBA_SCHEDULER_JOBS`:

```sql
SELECT job_name, enabled FROM DBA_SCHEDULER_JOBS WHERE job_name = 'COLETA_STATS';
```

Você também pode monitorar a execução do job consultando a view `DBA_SCHEDULER_JOB_RUN_DETAILS`:

```sql
SELECT * FROM DBA_SCHEDULER_JOB_RUN_DETAILS WHERE job_name = 'COLETA_STATS' ORDER BY log_date DESC FETCH FIRST 10 ROWS ONLY;
```

Lembre-se de ajustar os parâmetros conforme necessário para atender aos requisitos do seu ambiente.

#**OUTRA FORMA: Para criar um job no Oracle Scheduler que execute o procedimento `statspack.snap` a cada 3 minutos, você pode seguir este exemplo:**

```sql
BEGIN
  DBMS_SCHEDULER.create_job (
    job_name        => 'COLETA_STATS',
    job_type        => 'PLSQL_BLOCK',
    job_action      => 'BEGIN statspack.snap; END;',
    start_date      => SYSTIMESTAMP,
    repeat_interval => 'FREQ=MINUTELY; INTERVAL=3',
    enabled         => TRUE
  );
END;
/
```

Neste exemplo, o job é criado com o nome `COLETA_STATS`, e o procedimento `statspack.snap` é executado a cada 3 minutos. O parâmetro `repeat_interval` é configurado para `FREQ=MINUTELY` para especificar a frequência de execução por minutos e `INTERVAL=3` para indicar que o intervalo entre as execuções é de 3 minutos.

Após a execução bem-sucedida desse script, o job será criado e iniciará a coleta de estatísticas com o Statspack a cada 3 minutos. Certifique-se de que o job esteja habilitado para que seja executado conforme configurado.